In [88]:
import copy
import numpy as np
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.utilities.risk_reporting import createValueReport

In [89]:
IndexManager.instance()

In [90]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [91]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [92]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]

In [93]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [94]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    strike=95.7,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

a = ve.value[1]
print("Baseline PV (a):", a)


RFR Future PV: ['USD', np.float64(3.2333518462344694e-07)]
Baseline PV (a): 3.2333518462344694e-07


In [95]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([99999999.99999993,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [96]:
yc.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  -98.924743  ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  -97.89682633,    0.        ,
           0.        ,    0.        , 

In [97]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, rfr_future, request="all", space="pv")

In [98]:
report['param_risk']

array([-1000000.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.])

In [99]:
report = createValueReport(vp, yc, rfr_future, request="all", space="quote")

In [100]:
report['quote_risk']

array([989247.43001761,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ])

V = df(0, T + 3M) (Fut - K)

dV/dFut = df 

dV/ddf = (Fut - K)

risk a disc * (Fut - K)

In [101]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B",to_date="2026-06-24")

In [102]:
yc.gradient_

array([-0.24489546, -0.2422043 , -0.24758662, -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ])

In [103]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.71],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [104]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [105]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [106]:
yc2 = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [107]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    strike=95.7,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc2,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

b = ve.value[1]

RFR Future PV: ['USD', np.float64(-9892.721677124064)]


In [108]:
a

np.float64(3.2333518462344694e-07)

In [109]:
b

np.float64(-9892.721677124064)

In [110]:
(a-b)/0.01

np.float64(989272.1677447398)

In [111]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2029-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.0369,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(372.185462544061)] Par Rate: 0.036800000000000214


In [112]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([-252871.85799192, -250093.04636564, -251009.19072718,
       -251009.19072718, -243708.83747071, -241030.71837763,
       -241927.50800688, -241927.50800688, -234897.83162923,
       -234897.83162923, -233159.20051068, -233159.20051068,
       -899703.82442526,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [113]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, ois_swap, request="all", space="quote")

In [114]:
report['param_risk']

array([ 7.52863442e-01,  7.52683500e-01,  6.52516444e-01,  6.59353366e-01,
        5.33190664e-01,  5.32556694e-01,  4.24645285e-01,  4.28725654e-01,
        3.02391082e-01,  3.05158128e-01,  1.85720246e-01,  1.87381411e-01,
        1.00007435e+06, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [115]:
report['quote_risk']

array([-7.44768225e-01, -7.36853258e-01, -6.32169162e-01, -6.32329081e-01,
       -5.06320218e-01, -5.00909467e-01, -3.95609336e-01, -3.95689463e-01,
       -2.76559148e-01, -2.76516255e-01, -1.66796803e-01, -1.66785392e-01,
        3.72213134e+06, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [116]:
swap = ProductOvernightSwap(
    effectiveDate="2026-09-24",
    maturityDate="2027-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.0369,
    notional=1000000,
    position="LONG"
)

In [117]:
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

a= ve.value[1]

OIS Swap PV: ['USD', np.float64(-1213.9488034361566)] Par Rate: 0.03818510951731962


In [118]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([    306.85928087,     303.48720086,     310.23136088,
           310.23136088, -242110.55788225, -239450.00230113,
       -240311.66490646, -240311.66490646,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [119]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, swap, request="all", space="pv")

In [120]:
report['param_risk']

array([  -3.06859281,   -3.06785938,   -3.16896239,   -3.20216608,
       2524.5780404 , 2521.57628937, 2554.94444223, 2579.49461949,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ])

In [121]:
report = createValueReport(vp, yc, swap, request="all", space="quote")
report['quote_risk']

array([    3.03559755,     3.00333697,     3.07014531,     3.07092196,
       -2397.3504991 , -2371.73140156, -2380.24513801, -2380.72723207,
           0.        ,     0.        ,     0.        ,     0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ])

In [122]:
MARKET_DF_3 = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.09],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [123]:
data_objs_3, dc_3 = build_yc_data_collection(MARKET_DF_3)

In [124]:
build_methods_3 = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [125]:
yc3 = YieldCurve("2025-09-24", dc_3, build_methods_3)
print("Curve components:", yc3.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [126]:
ve = ValuationEngineRegistry().new_valuation_engine(
    yc3, {"FUNDING INDEX": "SOFR-1B"}, swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(-1189.9746983791483)] Par Rate: 0.03815969853853918


In [127]:
b = ve.value[1]

In [128]:
a

np.float64(-1213.9488034361566)

In [129]:
b

np.float64(-1189.9746983791483)

In [130]:
(a-b)/0.01

np.float64(-2397.4105057008273)